## Model Describer Meetup Tutorial

In this notebook, we will be doing some brief EDA of the [bicycle trip dataset](https://www.kaggle.com/pronto/cycle-share-dataset/home). This is data from the Pronto Cycle Share system which consists of 500 bikes and 54 stations located in Seattle. 

The key question for this tutorial will be whether or not there are noticeable differences in trip duration by gender and by user age. We will not be controlling for location information (a known deficit of this tutorial). 

In this tutorial, we will be covering the following:

* [Data prep](#prep)
* [Exploratory data analysis](#eda)
* [Build neural network model](#neural)
* [Model Describer Evaluation](#mdesc_regression)
* [Model Describer Sensitivity](#mdesc_sensitivity_regression)
* [Additional thoughts](#thoughts)

In [1]:
import os
from datetime import datetime
import warnings

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import keras
import pandas as pd

from mdesc.models import (Eval, ClassifierEval, Sensitivity, ClassifierSensitivity)

/Users/jasonlewris/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning:

Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.

Using TensorFlow backend.


In [2]:
# initialize plotly notebook_mode
init_notebook_mode(connected=True)

# filter out warning messages
warnings.filterwarnings('ignore')

#### Data Prep <a id='prep' />

Read in data and perform basic data manipulations

In [3]:
# read in the trip and weather data. We will not be using the station data, but there are a number of ways we could use
# this if we took advantage of the location information. 
base_path = '/Users/jasonlewris/Desktop/cycle-share-dataset/'
# base_path = r'C:\Users\jlewris\Desktop\BikeData'
trip = pd.read_csv(os.path.join(base_path, 'trip.csv'), error_bad_lines=False)
weather = pd.read_csv(os.path.join(base_path, 'weather.csv'))

b'Skipping line 50794: expected 12 fields, saw 20\n'


In [4]:
# for the purposes of this tutorial, we are going to drop rows without demographic data
trip = trip.loc[trip['gender'].notnull()]

In [5]:
# some basic preprocessing functions that should be pulled out into a utility file

def convert_date(dte):
    """
    convert string date into datetime object
    
    Parameters
    ----------
    dte - string
          datetime string object
          
    Return 
    ----------
    datetime obj
        string input converted to datetime object
    """
    try:
        dte = datetime.strptime(dte, '%m/%d/%Y %H:%M')
    except ValueError:
        dte = datetime.strptime(dte, '%m/%d/%Y')
    return dte

def return_part_date(dte, part_of_date='month'):
    """
    Pull the part_of_date from input datetime object
    
    Parameters
    ----------
    dte - datetime object
          input datetime object
    
    part_of_date - str - ['month', 'day', 'year', 'hour', 'minute']
          
    Return 
    ----------
    part of date
        part of date, i.e. year, hour, etc. 
    """
    dte = convert_date(dte)
    return getattr(dte, part_of_date)

def is_weekday(dte):
    """
    return whether a dte is a weekday or not
    
    Parameters
    ----------
    dte - datetime object
          input datetime object
              
    Return 
    ----------
    binary flag
        1 if is weekday else 0 
    """
    dte = convert_date(dte)
    if dte.weekday() not in [5, 6]:
        return 1
    else:
        return 0
    
def map_hours(hr):
    """
    map input hour into bin
    
    Parameters
    ----------
    hr - int - required
          input hour
              
    Return 
    ----------
    string
        bin of hour (early_morning, morning_rush, mid_day, evening_rush, evening)
    """
    if hr < 6:
        return 'early_morning'
    elif hr >= 6 and hr < 10:
        return 'morning_rush'
    elif hr >= 10 and hr < 15:
        return 'mid_day'
    elif hr >= 15 and hr < 19:
        return 'evening_rush'
    else:
        return 'evening'

In [6]:
# pull relevant parts of date
trip['start_day'] = trip['starttime'].apply(lambda x: return_part_date(x, part_of_date='day'))
trip['start_year'] = trip['starttime'].apply(lambda x: return_part_date(x, part_of_date='year'))
trip['start_month'] = trip['starttime'].apply(lambda x: return_part_date(x, part_of_date='month'))
trip['start_hour'] = trip['starttime'].apply(lambda x: return_part_date(x, part_of_date='hour'))
weather['start_day'] = weather['Date'].apply(lambda x: return_part_date(x, part_of_date='day'))
weather['start_year'] = weather['Date'].apply(lambda x: return_part_date(x, part_of_date='year'))
weather['start_month'] = weather['Date'].apply(lambda x: return_part_date(x, part_of_date='month'))

# test if date is weekday or not
trip['weekday'] = trip['starttime'].apply(lambda x: is_weekday(x))

In [7]:
# pull out just the mean values for weather
weather_sub = weather[['Mean_Temperature_F', 'MeanDew_Point_F', 'Mean_Humidity', 
                      'Mean_Visibility_Miles', 'Mean_Wind_Speed_MPH', 'Precipitation_In', 
                      'start_day', 'start_year', 'start_month']]

In [8]:
trip = pd.merge(trip, weather_sub, on=['start_day', 'start_year', 'start_month'], how='left')

In [9]:
# drop end location information
trip = trip.drop(['to_station_name', 'to_station_id', 'from_station_name'], axis=1)

In [10]:
# get age of rider
trip['rider_age'] = trip['start_year'] - trip['birthyear']

### Basic Exploratory Data Analysis <a id='eda' />

Perform basic exploratory analysis of the bicycle data. Some preliminary analysis includes:

* Number of trips by weekend/weekday by hour
* Average trip duration by weekend/weekday by hour
* Average trip duration by gender
* Correlation heatmap of key variables of interest

In [11]:
# number of trips by day type and hour
numtrips_weekday = trip.groupby(['weekday', 'start_hour'])['trip_id'].nunique().reset_index(name='numTrip')

total_weekday_trips = trip.loc[trip['weekday'] == 1]['trip_id'].nunique()
total_weekend_trips = trip.loc[trip['weekday'] == 0]['trip_id'].nunique()

numtrips_weekday['percentTrips'] = numtrips_weekday.apply(lambda x: x['numTrip']/total_weekday_trips if x['weekday'] == 1 else x['numTrip']/total_weekend_trips, axis=1)

In [12]:
# get number of trips by hour of day

weekday = numtrips_weekday['weekday'] == 1
weekend = numtrips_weekday['weekday'] == 0

trace1 = go.Bar(
    x = numtrips_weekday.loc[weekday, 'start_hour'].tolist(),
    y = numtrips_weekday.loc[weekday, 'percentTrips'].tolist(), 
    name='Weekday Trips'
)

trace2 = go.Bar(
    x = numtrips_weekday.loc[weekend, 'start_hour'].tolist(),
    y = numtrips_weekday.loc[weekend, 'percentTrips'].tolist(), 
    name='Weekend Trips'
)

data = [trace1, trace2]

layout = go.Layout(barmode='group', 
                  title='Weekend vs. Weekday Trips by Hour',
                  xaxis=dict(title='Hour'), 
                  yaxis=dict(title='Percent of Trips'))

fig = go.Figure(data=data, layout=layout)

iplot(fig)

In [13]:
# average trip duration by day type by hour
trip_dist = trip.groupby(['weekday', 'start_hour'])['tripduration'].mean().reset_index(name='tripDist')

In [14]:
trace1 = go.Scatter(
    x = trip_dist.loc[weekday, 'start_hour'].tolist(),
    y = trip_dist.loc[weekday, 'tripDist'].tolist(), 
    name='Weekday Trips', 
    mode='lines'
)

trace2 = go.Scatter(
    x = trip_dist.loc[weekend, 'start_hour'].tolist(),
    y = trip_dist.loc[weekend, 'tripDist'].tolist(), 
    name='Weekend Trips', 
    mode='lines'
)

data = [trace1, trace2]

layout = go.Layout(
                  title='Weekend vs. Weekday Trip Duration by Hour',
                  xaxis=dict(title='Hour'), 
                  yaxis=dict(title='Trip Duration (seconds)'))

fig = go.Figure(data=data, layout=layout)

iplot(fig)

In [15]:
# we can clearly see peaks during the weekday during rush hour. Lets map these to reduce our 
# total feature exposure

trip['start_hour'] = trip['start_hour'].apply(lambda x: map_hours(x))

In [16]:
tripgender = trip.groupby('gender')['tripduration'].mean().reset_index(name='tripDist')

data = [go.Bar(
    x=tripgender['gender'].tolist(),
    y=tripgender['tripDist'].tolist()
)]

layout = go.Layout(
                  title='Average Trip Duration (seconds) by Gender',
                  xaxis=dict(title='Gender'), 
                  yaxis=dict(title='Trip Duration (seconds)'))

fig = go.Figure(data=data, layout=layout)

iplot(fig)

In [17]:
corr = trip[['rider_age', 'Mean_Temperature_F', 'Mean_Humidity', 
             'tripduration', 'weekday']].corr()

trace = go.Heatmap(z=corr.values.tolist(), 
                   x=corr.index.tolist(),
                   y=corr.columns.tolist())

layout = go.Layout(
                  title='Correlation Heatmap',
    )

data = [trace]

fig = go.Figure(data=data, layout=layout)

iplot(fig)

### Build Neural Network Model <a id='neural' />

Build out example neural network in keras to predict trip duration

In [18]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD, Adam
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np

In [19]:
# for the purposes of this tutorial, we are dropping the gender category other due to lack of observations. 
# more time could be spent doing a better sampling method for training to include sparse categories

# drop certain columns 
trip_model = trip.drop(['stoptime', 'bikeid', 'trip_id', 'from_station_id',
                       'starttime', 'birthyear', 'start_day'], axis=1)



X = trip_model.loc[:, trip_model.columns != 'tripduration']
y = trip_model.loc[:, 'tripduration'].values.reshape((X.shape[0], 1))

In [20]:
# map hours
X['weekday'] = X['weekday'].replace({0: 'weekend',
                                    1: 'weekday'})

In [21]:
# pull out categories of interest
categories = ['usertype', # 'from_station_id', 
              'gender', 'start_year', 
              'start_month', 'start_hour', 'weekday']

continuous = [col for col in X.columns.tolist() if col not in categories]

In [22]:
# fill missing of infite values with the mean value of the column
for cont in continuous:
    X[cont] = X[cont].fillna(X[cont].mean())

In [23]:
# create binary values for categories 
X_cat = pd.get_dummies(X[categories], columns=categories)

X_cat_cols = X_cat.columns.tolist()

In [24]:
from sklearn.preprocessing import normalize

# lets help out our neural network model by squeezing in the range of possible values

# you should do this for your target values as well, but for the purposes of this tutorial
# and model-describer, we will be leaving it out

X_cont = normalize(X[continuous])

In [25]:
# concatenate the categorical and continuous arrays back for model training
X = np.hstack((X_cat, X_cont))

feature_names = X_cat_cols + continuous

In [26]:
# key note is making sure the indexes for whatever groupby variables you want to include are maintained 
# with the splitting of the dataset. Here we will pass in the original data, and then hand select
# which groupby variables we want to use for model-describer

X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(X, y, trip, 
                                                   test_size=0.1, 
                                                   random_state=42)

In [28]:
# create a relatively simple neural network for regression purposes

model = Sequential()
model.add(Dense(512, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1, activation='linear'))

# slow learning rate - continued drop in validation loss gives evidence that
# many more epochs might be sufficient - also installing tensorflow with GPU backend
# would dramatically speed up training
adam = Adam(lr=0.0005)

model.compile(loss='mae', optimizer=adam)
history = model.fit(X_train, y_train, epochs=500, verbose=0, batch_size=256, validation_split=0.2)

In [29]:
# from the keras history callback, extract the specific loss values needed to visualize
loss = history.history['loss']
val_loss = history.history['val_loss']
num_epochs = [idx for idx, _ in enumerate(loss)]

trace1 = go.Scatter(
    x = num_epochs,
    y = loss, 
    name='Training Loss', 
    mode='lines',
    line=dict(shape='spline')
)

trace2 = go.Scatter(
    x = num_epochs,
    y = val_loss, 
    name='Validation Loss', 
    mode='lines',
    line=dict(shape='spline')
)

data = [trace1, trace2]

layout = go.Layout(
                  title='Training/Validation Loss',
                  xaxis=dict(title='Epoch Number'), 
                  yaxis=dict(title='Mean Absolute Error'))

fig = go.Figure(data=data, layout=layout)

iplot(fig)

### Model Describer Evaluation <a id='mdesc_regression' />

Fitting model describer evaluation function is easy. Simply provide the models prediction function directly, 
and feed in X, and y. The remainder of the parameters are for beuatification of outputs. 

The most straight forward instantiation is with no beutification and no groupby dataframe:

```python
from mdesc.models import Eval

RE = Eval(prediction_fn=model.predict)

res = RE.fit_transform(X=X, y=y)
```

You can easily see how your model is performing directly within your data science workflow using the following:

```python
# helper to remember what groupby_names have been set
RE.data_set.groupby_names
# Index(['gender', 'usertype', 'start_hour'], dtype='object')

RE.data_set.viz_now(groupby_name='gender')
```

In [30]:
# capture relevant groupby values
groupby_df = group_test[['gender', 'start_hour']]

In [31]:
# fitting model describer evaluation function is easy. Simply provide the models prediction function directly, 
# and feed in X, and y. The remainder of the parameters are for beuatification of outputs
RE = Eval(prediction_fn=model.predict, target_names='transit_time', 
                feature_names=feature_names)

res = RE.fit_transform(X=X_test, y=y_test, 
                       groupby_df=groupby_df)

return_group_array group_col: gender, group_level: Female
return_group_array group_col: gender, group_level: Male
return_group_array group_col: gender, group_level: Other
return_group_array group_col: start_hour, group_level: early_morning
return_group_array group_col: start_hour, group_level: evening
return_group_array group_col: start_hour, group_level: evening_rush
return_group_array group_col: start_hour, group_level: mid_day
return_group_array group_col: start_hour, group_level: morning_rush


In [36]:
RE.data_set.viz_now(groupby_name='gender')

### Model Describer Sensitivity <a id='mdesc_sensitivity_regression' />

Model describer sensitivity is also easy to instantiate. However, with sensitivity, we want to get an even better idea of how our model will perform on synthesized data that could possibly come into the pipeline after deployed into production. We can test the boundaries of this by setting the std parameter to nudge continuous data by x amount of standard deviations.  

Without any naming of columns or y levels, you can instantiate Sensitivity with the following:

```python
from mdesc.models import Sensitivity

RE = Sensitivity(prediction_fn=model.predict, std=0.5)

res = RE.fit_transform(X=X, y=y)
```

You can easily see how your model is performing directly within your data science workflow. Exact same calling method as with Eval:

```python
# helper to remember what groupby_names have been set
RE.data_set.groupby_names
# Index(['gender', 'usertype', 'start_hour'], dtype='object')

RE.data_set.viz_now(groupby_name='gender')
```

In [33]:
RE = Sensitivity(prediction_fn=model.predict, target_names='transit_time (seconds)', 
                feature_names=feature_names, std=0.5)

res = RE.fit_transform(X=X_test[0:10000], y=y_test[0:10000], 
                       groupby_df=groupby_df[0:10000])

return_group_array group_col: gender, group_level: Female
return_group_array group_col: gender, group_level: Male
return_group_array group_col: gender, group_level: Other
return_group_array group_col: start_hour, group_level: early_morning
return_group_array group_col: start_hour, group_level: evening
return_group_array group_col: start_hour, group_level: evening_rush
return_group_array group_col: start_hour, group_level: mid_day
return_group_array group_col: start_hour, group_level: morning_rush


In [34]:
RE.data_set.viz_now(groupby_name='gender')

### Additional Thoughts <a id='thoughts' />

We have reviewed how to use model-describer evaluation and sensitivity functionality and interact with the results directly in a data scientist workflow. Reviewing the approach above, there are a number of improvements we could make:

* It is clear in the 'other' gender category that the model does not have enough data. A better sampling approach would prove quite useful here
* The model needs more training time and would yield value to incorporate cross validation
* It is clear that the model has inherited striking differences between men and women for the targeted output. This needs to be reviewed based on your mission requirements and whether or not you need to control to remove these discrepancies

**Most importantly** we want your feedback! If you have any ideas on how to boost your workflows by using this package, please let us know. We want to make a generalized solution that has real utility in data science at work. Please post an issue [here](https://github.com/DataScienceSquad/model-describer/issues) with a detailed explanation of your idea and we will be happy to examine it. We are also actively looking for contributors to the package, so please let us know if you have an interest in developing new features. 

To get in touch with Daniel or myself, please review contact information below:

**Jason Lewris**: [LinkedIn](https://www.linkedin.com/in/jasonlewris/); [Github](https://github.com/datawrestler); [Stack](https://stackoverflow.com/users/4941992/datawrestler); [Email](mailto:jlewris@deloitte.com)

**Daniel Byler**: [LinkedIn](https://www.linkedin.com/in/danielbyler/); [Github](https://github.com/danielbyler); [Email](mailto:dbyler@deloitte.com)